Amazon EC2 スポットインスタンスを利用してVCノードを起動します。

**スポットインスタンスとは？**
- 通常のオンデマンドインスタンスと比較して低コスト
  * AWSの余剰リソースが使われ、最大90%割引
- 変動する価格に対して入札方式によりインスタンス利用料金が決まる
  * スポット価格 <= 入札価格 ... 指定したインスタンスが起動
  * 入札価格 < スポット価格  ... インスタンスが終了される

**本Notebookは、[101-VCノードの起動、削除](./101-VCノードの起動、削除.ipynb) にスポットインスタンス向けの設定内容を追記したものです。**
- スポットインスタンス向けの設定箇所は、セクションのタイトル末尾に★マークを付けています。
  * 2.5.1 flavor★
  * 2.5.2 specオブジェクトを作成する★
  * 2.5.3 AWSスポットインスタンス専用のspec設定★

# VCノードの起動、削除

このNotebookではVCP SDKを利用してVCノードを起動、削除する手順を記しています。

## 概要

![構成](images/101-001.png)

### 用語説明

上の図、またはこれ以降に示す図に記されている構成要素についての簡単な説明を以下に記します。

* [VPC](https://aws.amazon.com/jp/vpc/)
  - Amazon が提供している AWS 内の仮想プライベートネットワーク環境
* [Amazon EC2](https://aws.amazon.com/jp/ec2/)
  - Amazon が提供している仮想コンピューティング環境
* [Azure Virtual Network](https://azure.microsoft.com/ja-jp/services/virtual-network/)
  - Microsoft が提供しているクラウド内の仮想プライベートネットワーク環境
* [Azure Virtual Machine](https://azure.microsoft.com/ja-jp/services/virtual-machines/)
  - Microsoft が提供している仮想コンピューティング環境
* VC Controller
  - VCPがユーザに対して提供しているサービス群
* VC Controller Core
  - ユーザからのAPI呼び出しを受けとりUnitGroupの作成、削除などを行う
* Grafana
  - 可視化ツール、ダッシュボードツール
  - VCP では VCノード のモニタリング状況を表示するのに利用している
* VCP SDK
  - VCPの機能を呼び出して VM/BM の作成、削除などを行うPython3のライブラリ
* VCノード
  - Amazon EC2, Azure Virtual Machine などの計算資源を抽象化したVCPのノード
* Unit
  - 同質（同じ計算資源(cpu, memory, ...)、同じクラウド、同じ用途、...）であるVCノードをまとめて扱うためのもの
* UnitGroup
  - 複数のUnitをまとめて扱うためのものです

## 操作手順

大まかな操作手順は以下のようになります。

1. VCP SDKを用いてVCノード(Amazon EC2 インスタンス)を起動する
1. 起動したVCノードに ssh でログインして操作する
1. GrafanaでVCノードの利用状況を確認する
1. Unit内のVCノードをスケールアウト、スケールインする
1. 別のパブリッククラウド(Microsoft Azure)のVCノードを起動する
1. 起動したVCノードをすべて削除する

# VCノード(Amazon EC2インスタンス)の起動

VCP SDKを用いてVCノード(Amazon EC2インスタンス)を起動します。また、起動したVCノードに対してsshでログインして操作を行います。

![EC2の起動](images/101-003.png)

## アクセストークンの入力

VCP SDKを利用するにはVC Controllerのアクセストークンが必要となります。次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [1]:
from getpass import getpass
vcc_access_token = getpass()

········


## VCP SDKの初期化

VCP SDKの初期化を行います。

In [2]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

# VCP SDKの初期化
vcp = VcpSDK(
    vcc_access_token,  # VCCのアクセストークン
)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
2019-XX-XX XX:XX:XX,XXX - ERROR - config vc failed: http_status(403)
2019-XX-XX XX:XX:XX,XXX - ERROR - 2019/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

この場合はアクセストークンの入力からやり直してください。

`vcp`からは UnitGroup の管理（作成、情報取得）や VCP SDK自体に関する情報取得を行うことができます。例えば、次のセルを実行するとUnitGroupの一覧が表示されます。

> まだUnitGroupを作成していないので、ヘッダー以外はなにも表示されないはずです。

In [3]:
# UnitGroupの一覧を DataFrame で表示する
vcp.df_ugroups()

,vcno,vctype,vcname,state,vcid,cdate


次のセルを実行すると VCP SDK と VC Controller のバージョンが表示されます。

In [4]:
vcp.version()


vcplib:
  filename: /home/jovyan/vcpsdk/vcplib/occtr.py
  version: 20.10.0+20201001

vcpsdk:
  filename: /home/jovyan/vcpsdk/vcpsdk/vcpsdk.py
  version: 21.04.0+20210401

  plugin:
    aws: 1.2+20191001
    aws_disk: 1.0+20190408
    aws_spot: 1.1+20191001
    azure: 1.2+20191001
    vmware: 1.1+20191001
    azure_disk: 1.0+20190408
    sakura: 1.1+20191001
    sakura_disk: 1.0+20190930
    oracle: 1.0+20200331
    oracle_disk: 1.0+20200331
    aic: 1.2+20191001
    abc: 1.3+20190408
    hokudai: 1.1+20191001
    chameleon: 1.0+20200831
    chameleon_ext: 20200831
    gcp: 1.0+20190408
    onpremises: 1.0+20190408

vc_controller:
  host: localhost
  name: vcc511
  wait_timeout_sec: 1000(default 15min)
  vc_controller: 21.04.0+20210401
  vc_controller_git_tag: 21.04.0-117-gd0b1d69-dirty

  plugin:
    aic: 1.2+20210401
    hokudai: 1.1+20210401
    vmware: 1.1+20210401
    sakura: 1.1+20210401
    aws: 1.2+20210401
    azure: 1.2+20210401
    oracle: 1.0+20210401
    abc: 1.3+20210401


## UnitGroupの作成

`handson101`という名前の UnitGroup を作成します。

In [5]:
# UnitGroupの作成
unit_group = vcp.create_ugroup(
    'handson101'                   # UnitGroupの名前
)

作成したUnitGroupの状態を表示してみます。

> UnitGroup の名前が `handson101` であることと、その状態が `RUNNING` であることなどが確認できるはずです。

In [6]:
print(unit_group)

[Vc]
+ type[compute] name[handson101] vcno(5) state[RUNNING] vcid[8928ee1425c84452876aa39cbf948b26]



UnitGroupの一覧を表示してみます。

In [7]:
# UnitGroupの一覧を DataFrame で表示する
vcp.df_ugroups()

,vcno,vctype,vcname,state,vcid,cdate
0,5,compute,handson101,RUNNING,8928ee14...,2021/07/05 06:45:38 UTC


## UnitGroupの状態を確認する

UnitGroupに属している Unit の一覧を表示します。

> まだ、このUnitGroupでは Unit を作成していないのでヘッダー以外は何も表示されません。

In [8]:
# Unitの一覧を DataFrame で表示する
unit_group.df_units()

,vctype,vcno,vcname,vc_state,vcid,cdate,unit_name,unit_state


UnitGroupに属しているVCノードの一覧を表示します。

> まだVCノードが存在していないので何も表示されません。

In [9]:
unit_group.df_nodes()

[]

## VCノードのspecを指定する

Unitを構成するVCノードがどのような設定であるかを指定するためのオブジェクトとしてVCP SDKでは`spec`オブジェクトを用意しています。VCP SDKの利用者は`spec`オブジェクトのプロパティを設定することで Unitに起動するVCノードの設定内容を定義することができます。

`spec`オブジェクトの設定項目の例を以下に示します。

* 仮想マシンのインスタンスタイプ
  - m5.large, c5.large, ...
* 仮想マシンのルートボリュームサイズ(GiB)
* 仮想マシンに割り当てるプライベートIPアドレス
* Unit内に作成するVCノードの数

設定できる項目はクラウドプロバイダ(aws, azure, ...)毎に異なります。

### flavor ★

`spec`オブジェクトの全てのパラメータを毎回設定するのは煩雑になるので典型的な構成のパラメータセットを事前に定義しています。事前に定義した`spec`パラメータセットのことを VCP SDKでは `flavor`と呼んでいます。`spec`に設定できるパラメータはクラウドプロバイダ毎に異なるので `flavor`の定義もクラウドプロバイダ毎に行っています。

次のセルを実行すると aws用に定義している `flavor` の一覧が表示されます。

In [10]:
# スポットインスタンスを利用するためには、aws を aws_spot に変更
vcp.df_flavors('aws_spot')

,flavor,instance_type,volume_type,volume_size
0,small,m4.large,standard,8
1,medium,m4.xlarge,standard,40
2,large,m4.2xlarge,standard,100


### spec オブジェクトを作成する ★

specオブジェクトを作成します。specオブジェクトを作成するにはプロバイダと`flavor`を指定します。ここでは以下の値を指定します。

* プロバイダ: `aws_spot`
* flavor: `small`

In [11]:
# スポットインスタンスを利用するためには、aws を aws_spot に変更
spec = vcp.get_spec('aws_spot', 'small')

作成した`spec`の設定内容を確認してみます。

> `instance_type`パラメータに `flavor` の指定と対応するEC2インスタンスタイプ`m4.large`が設定されていることが確認できるはずです。
> `volume_size`, `volume_type`についてもそれぞれ`flavor`と対応する値が設定されます。

In [12]:
print(spec)


aws_spot
------------------------

aws_spot
------------------------

    cci_version: "1.1"
    name: 
    description: 
    private_network: default
    num_nodes: 1
    image: vcp/base:1.6.2
    recovery_mode: none
    docker_opt: {}
    params:
        {'v': [], 'e': []}

    instance_type: m4.large
    volume_size: 8
    volume_type: standard
    volume_id: []
    cloud_image: default
    ip_address_list: []
    tags: {}
    spot_type: one-time
    spot_price: 
    block_duration_minutes: 0
    instance_type: m4.large
    instance_type: m4.large
    volume_size: 8
    volume_type: standard
    volume_id: []
    cloud_image: default
    tags: {}


`spec`に対しては `flavor`で指定するだけではなく個々のパラメータを直接指定することもできます。例えば、以下のようなものが指定できます。

* num_nodes
  - Unit内に作成するVCノードの数: デフォルト=1
* ip_addresses
  - VCノードに割り当てるプライベートIPアドレスのリスト
  - このパラメータを指定しない場合は利用可能なアドレスが自動的に割り当てられる
* instance_type
  - Amazon EC2のインスタンスタイプ
  - flavorで設定されているもの以外を利用したい場合に指定する
* volume_size
  - Amazon EC2インスタンスのルートボリュームに割り当てるサイズ(GiB)
  - flavorで設定されているもの以外を利用したい場合に指定する
* volume_type
  - Amazon EC2インスタンスのルートボリュームのEBSタイプ
  - flavorで設定されているもの以外を利用したい場合に指定する

試しに `volume_size`を指定してみます。

In [13]:
spec.volume_size = 15

### AWSスポットインスタンス専用のspec設定 ★

AWSスポットインスタンスに対して、以下のパラメータを指定することができます。

- インスタンス1時間当たりの上限価格(=入札価格)
  * 米ドル単位で指定。指定無しの場合はオンデマンドインスタンスの価格が上限価格として指定される。
  > *小さすぎる価格では起動できない可能性がある*
- インスタンスの継続時間
  * 1〜6時間の範囲で 60, 120, 180, 240, 300, 360 のように分単位で指定可能
  > *長過ぎる継続時間では起動できない可能性がある*

In [14]:
# スポットインスタンスの1時間当たりの上限価格 (単位: USD)
spec.spot_price = '0.15'

# スポットインスタンスの継続時間 (単位: 分、60の倍数)
# spec.block_duration_minutes = 300

# インスタンスタイプ (0.15 USD 以内の例: m4.large, c4.large, t2.large, t2.medium)
# spec.instance_type = 'm4.large'

指定した値が`spec`の設定に反映されていることを確認してみます。

In [15]:
print(spec)


aws_spot
------------------------

aws_spot
------------------------

    cci_version: "1.1"
    name: 
    description: 
    private_network: default
    num_nodes: 1
    image: vcp/base:1.6.2
    recovery_mode: none
    docker_opt: {}
    params:
        {'v': [], 'e': []}

    instance_type: m4.large
    volume_size: 15
    volume_type: standard
    volume_id: []
    cloud_image: default
    ip_address_list: []
    tags: {}
    spot_type: one-time
    spot_price: 0.15
    block_duration_minutes: 0
    instance_type: m4.large
    instance_type: m4.large
    volume_size: 15
    volume_type: standard
    volume_id: []
    cloud_image: default
    tags: {}


`volume_size`の値が指定した値に変更されています。

### sshの鍵ファイルを設定する

VCノードにsshでログインするためには事前に公開鍵認証の鍵を登録する必要があります。そのための設定をここで行います。

VCノードに登録する公開鍵認証の**公開鍵**のパスを次のセルで指定してください。

In [16]:
import os
ssh_public_key = os.path.expanduser('~/.ssh/id_rsa.pub')

指定した公開鍵を `spec` に設定します。

In [17]:
spec.set_ssh_pubkey(ssh_public_key)

sshの公開鍵に関する設定が`spec`に反映されたことを確認してみます。次のセルを実行すると`params`の`e`の`AUTHORIZED_KEYS`に値が設定されていることが確認できます。

In [18]:
print(spec)


aws_spot
------------------------

aws_spot
------------------------

    cci_version: "1.1"
    name: 
    description: 
    private_network: default
    num_nodes: 1
    image: vcp/base:1.6.2
    recovery_mode: none
    docker_opt: {}
    params:
        {'v': [], 'e': ['AUTHORIZED_KEYS=c3NoLXJzYSBBQUFBQjNOemFDMXljMkVBQUFBREFRQUJBQUFCQVFDc09nTzdmTXJMSG5yRkpSZFROVkVOaWRDNlJTYlNYSE94dmxzSWUvMWc3Nk9IdXB5TE9EUUJ5aXhHc2t1QlVkaXBPaElGV3doeUR4RFRVMU9Xb1BLbHhuTndiV29oZXh1T2s3bmtCYmJ6Y09hT2hhZW5EL1RVdGVLeU05Tm85SW5PTktyS3lTQ0pQVklCUklTVlNXenF6ZzM4NENzSkNNVVBXdVNRcnZmNFpXajI3TDZCYUdaa0c2elVYaVVmMzAvZSsvcitrV2NIZFNFUUcyUHhqRk1iWU9aQkZSMzR5K1RuaWlpTE5LNFlXODgvajdFbXBCRGZ2VEtzL0ZLTXd3WjRjWkdqbmx6eWxFOEdldGhVNC9zcWFYQ1dZSWtHOFJzSjQ2MzRLNU5uTkdRSG80WEZPRzQ0TEJFdkhRTlJoajZCSU14eGd5TkgxVFZlUTdFT2RZNG4gdXNlcjUxMXhlNWIwNTJAMGVhNDIwMGNjMDUxCg==']}

    instance_type: m4.large
    volume_size: 15
    volume_type: standard
    volume_id: []
    cloud_image: default
    ip_address_list: []
    tags: {}
  

後でVCノードにログインする際にsshの秘密鍵も必要になるので、ここで設定しておきます。次のセルで**秘密鍵**のパスを指定してください。

In [19]:
ssh_private_key = os.path.expanduser('~/.ssh/id_rsa')

公開鍵と秘密鍵が正しいペアであることをチェックします。次のセルを実行してエラーにならないことを確認してください。

In [20]:
!grep -q "$(ssh-keygen -y -f {ssh_private_key})" {ssh_public_key}

<!---
他にどのようなパラメータが設定できるのかについては
「[VCPSDK APIドキュメント](/handson/files/DOC/index.html#module-vcpsdk.plugins.spec)」を参照してください。
--->

## Unitの作成とVCノードの起動

Unitを作成します。Unitを作成すると同時に VCノード（ここでは Amazon EC2インスタンス）が起動します。

> 処理が完了するまで1分半～2分程度かかります。

In [21]:
# Unitの作成（同時に VCノードが作成される）
unit = unit_group.create_unit(
    'aws-server',               # Unit名の指定
    spec
)

2021-07-05 15:47:13,576 - INFO - BOOTING ... 0 sec
2021-07-05 15:47:18,795 - INFO - BOOTING ... 5 sec
2021-07-05 15:47:23,952 - INFO - BOOTING ... 10 sec
2021-07-05 15:47:29,087 - INFO - BOOTING ... 15 sec
2021-07-05 15:47:34,228 - INFO - BOOTING ... 20 sec
2021-07-05 15:47:39,358 - INFO - BOOTING ... 25 sec
2021-07-05 15:47:44,507 - INFO - BOOTING ... 30 sec
2021-07-05 15:47:49,645 - INFO - BOOTING ... 35 sec
2021-07-05 15:47:54,803 - INFO - BOOTING ... 40 sec
2021-07-05 15:47:59,972 - INFO - BOOTING ... 45 sec
2021-07-05 15:48:05,156 - INFO - BOOTING ... 50 sec
2021-07-05 15:48:10,361 - INFO - BOOTING ... 55 sec
2021-07-05 15:48:15,537 - INFO - BOOTING ... 60 sec
2021-07-05 15:48:20,694 - INFO - BOOTING ... 65 sec
2021-07-05 15:48:25,879 - INFO - BOOTING ... 70 sec
2021-07-05 15:48:31,106 - INFO - BOOTING ... 75 sec
2021-07-05 15:48:36,277 - INFO - BOOTING ... 80 sec
2021-07-05 15:48:41,454 - INFO - BOOTING ... 85 sec
2021-07-05 15:48:46,608 - INFO - BOOTING ... 90 sec
2021-07-05 15:

UnitGroupに属しているUnitの一覧表示を行い、Unitが作成されていることを確認します。

In [22]:
# Unitの一覧を DataFrame で表示する
unit_group.df_units()

,vctype,vcno,vcname,vc_state,vcid,cdate,unit_name,unit_state
0,compute,5,handson101,RUNNING,8928ee14...,2021/07/05 06:45:38 UTC,aws-server,RUNNING


UnitGroupに属しているVCノードの一覧表示を行い、VCノードが起動していることを確認します。VCノードが正常に起動していることは `node_state` の表示が `RUNNING`になっていることで確認できます。

In [23]:
# VCノードの一覧を DataFrame で表示する
unit_group.df_nodes()

,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,5,handson101,aws-server,RUNNING,1,ff27f448...,RUNNING,172.30.2.104,i-0f3a952880fd13de9,VCP-8dbe841c-8928ee14,none


ここでは VCノードの起動が完了するまで待ち合わせるモードでUnitの作成を行いましたが、非同期処理でUnit, VCノードの作成を行うこともできます。`create_unit()`の`wait_for`パラメータに`False`を指定すると非同期モードでのUnit, VCノードの作成が行えます。

```
unit = unit_group.create_unit('aws-server', spec, wait_for=False)
```

非同期モードでUnitを作成した場合にUnitやVCノードが起動したことを確認するには、以下に示す方法で状態取得を行ってください。

1. `unit_group.df_units()`
  * Unitの一覧表示の`unit_state`の項目で Unitの状態を確認できます
  * 起動中は `BOOTING`、起動が完了して実行中になると `RUNNING` と表示されます
 
1. `unit.df_nodes()`, `unit_group.df_nodes()`
  * VCノードの一覧表示の`node_state`の項目で VCノードの状態を確認できます
  * 起動中は `BOOTING`、起動が完了して実行中になると `RUNNING` と表示されます

非同期モードでUnit作成を行った場合でも、以下のメソッドを利用することで後からVCノード起動の待ち合わせを行うことができます。
* `unit_group.wait_unit_applied(unit_name)`
  - unit_nameで指定されたUnitの状態が `RUNNING` または `ERROR`になるのを待ちます

## 疎通確認

起動した VCノードに対して`ping`を行ってみます。

まずUnitGroup内で起動しているVCノードに割り当てられているプライベートIPアドレスの値を取得して変数 `ip_address`に格納します。

In [24]:
# unit_group.find_ip_addresses() は UnitGroup内の全VCノードのIPアドレスのリストを返します
ip_address = unit_group.find_ip_addresses(node_state='RUNNING')[0] # 今は１つのVCノードのみ起動しているので [0] で最初の要素を取り出す

print(ip_address)

172.30.2.104


実際に `ping` を行ってみます。

> Codeセルで先頭に `!` をつけるとシェルコマンドが実行できます。また `{}`で囲むことで Python の変数参照やコードの実行に置き換えることができます。詳しくは「[IPython Documentation](https://ipython.readthedocs.io/en/stable/interactive/tutorial.html#system-shell-commands)」などを参照してください。

In [25]:
# NIIのハンズオン環境ではネットワーク構成の事情により本NotebookからVCノードへのping実行はできません。
# !ping -c 5 {ip_address}

# VCノードに ssh でログインして操作する

起動したVCノードに ssh でログインして操作を行います。

まず、ログインする前に ~/.ssh/known_hosts のホストキーを更新します。

In [26]:
!touch ~/.ssh/known_hosts
# ~/.ssh/known_hosts から古いホストキーを削除する
!ssh-keygen -R {ip_address}

# ホストキーの登録
!ssh-keyscan -H {ip_address} >> ~/.ssh/known_hosts

Host 172.30.2.104 not found in /home/jovyan/.ssh/known_hosts


`ls -la` を実行してみます。

In [27]:
!ssh -i {ssh_private_key} -l root {ip_address} /bin/ls -la

total 20
drwx------    1 root     root          4096 Oct  5  2020 .
drwxr-xr-x    1 root     root          4096 Jul  5 06:52 ..
drwxr-xr-x    1 root     root          4096 Oct  5  2020 .cache
drwxr-xr-x    3 root     root          4096 Oct  5  2020 .gem
drwx------    1 root     root          4096 Jul  5 06:52 .ssh


VCノードにsshでログインするにはsshの引数に以下の指定が必要となります。

* sshの identity ファイル(-i)
  - `spec`オブジェクトに設定した公開鍵に対応する秘密鍵
* ユーザ名(-l)
  - `root`
* ログイン先のIPアドレス

sshのオプションを毎回指定するのも煩雑なので変数に設定しておきます。

In [28]:
ssh_opts = f"-i {ssh_private_key} -l root"

VCノードに対して`uname -a`, `df -h` などを実行してみます。

In [29]:
!ssh {ssh_opts} {ip_address} uname -a

Linux ip-172-30-2-104 4.4.0-159-generic #187-Ubuntu SMP Thu Aug 1 16:28:06 UTC 2019 x86_64 Linux


In [30]:
!ssh {ssh_opts} {ip_address} df -h

Filesystem                Size      Used Available Use% Mounted on
overlay                  14.6G      3.3G     10.6G  24% /
tmpfs                     3.9G         0      3.9G   0% /sys/fs/cgroup
udev                      3.9G         0      3.9G   0% /dev
tmpfs                     3.9G         0      3.9G   0% /dev/shm
/dev/xvda1               14.6G      3.3G     10.6G  24% /etc/resolv.conf
/dev/xvda1               14.6G      3.3G     10.6G  24% /etc/hostname
/dev/xvda1               14.6G      3.3G     10.6G  24% /etc/hosts
/dev/xvda1               14.6G      3.3G     10.6G  24% /var/lib/docker
none                      3.9G         0      3.9G   0% /tmp


# GrafanaでVCノードの利用状況を確認する

VCPではVCノードの利用状況(CPU負荷、メモリ使用量、ネットワーク）を確認するためGrafanaのダッシュボードを提供しています。

## Grafanaへのログイン

以下のセルを実行して Grafana ダッシュボードを開いてください。

最初にログイン画面が表示されるので ユーザ名、パスワードにそれぞれ `admin`, `admin`を入力してください。 

In [31]:
vcc_ctr = vcp.vcc_info()['host']
http_host = vcc_ctr.split(':')[0]
grafana_url = "https://{}/grafana/d/vcp/vcp-metrics?refresh=5s".format(http_host)
print(grafana_url)

https://localhost/grafana/d/vcp/vcp-metrics?refresh=5s


![Grafanaのログイン画面](./images/grafana-login.png)

## 負荷状況のグラフ表示

以下のように表示されます。左半分のグラフがBaseコンテナの負荷状況で、右半分がアプリケーションコンテナの負荷状況のグラフです。

![負荷状況](./images/grafana-metrics.png)

VCノードに負荷をかけるために、アプリケーションコンテナで stress コマンドを実行します。

In [32]:
# 60秒間だけ CPU x 2 とメモリ 128 MB を消費する
!ssh {ssh_opts} {ip_address} \
    /usr/local/bin/docker run -td --rm --name stress-0 polinux/stress \
    stress --cpu 2 --io 1 --vm 2 --vm-bytes 128M --timeout 60s --verbose

Unable to find image 'polinux/stress:latest' locally
latest: Pulling from polinux/stress
63bc94deeb28: Pulling fs layer
8d43371bad93: Pulling fs layer
8d43371bad93: Verifying Checksum
8d43371bad93: Download complete
63bc94deeb28: Verifying Checksum
63bc94deeb28: Download complete
63bc94deeb28: Pull complete
8d43371bad93: Pull complete
Digest: sha256:b6144f84f9c15dac80deb48d3a646b55c7043ab1d83ea0a697c09097aaad21aa
Status: Downloaded newer image for polinux/stress:latest
7805aaa904344341ed02acee5bc7da7d8add7046b1056946f573f9b5cbe83c86


# 全てのVCノードを削除する

ここまで作成した全てのリソース（UnitGroup, Unit、VCノード）を削除します。この操作を行うことで AWS EC2インスタンスやAzure VMなどのクラウドに作成したリソースが削除されます。

> 全てのリソースの削除には 4～5分程度かかります。

In [33]:
unit_group.cleanup()

2021-07-05 15:53:35,508 - INFO - aws-server is DELETING ... 0 sec
2021-07-05 15:53:40,626 - INFO - aws-server is DELETING ... 5 sec
2021-07-05 15:53:45,775 - INFO - aws-server is DELETING ... 10 sec
2021-07-05 15:53:50,952 - INFO - aws-server is DELETING ... 15 sec
2021-07-05 15:53:56,137 - INFO - aws-server is DELETING ... 20 sec
2021-07-05 15:54:01,358 - INFO - aws-server is DELETING ... 25 sec
2021-07-05 15:54:06,574 - INFO - aws-server is DELETING ... 30 sec
2021-07-05 15:54:11,736 - INFO - aws-server is DELETING ... 35 sec
2021-07-05 15:54:16,863 - INFO - aws-server is DELETING ... 40 sec
2021-07-05 15:54:21,933 - INFO - cleanup completed. vc  is cleanup(no unit)


削除後の状態を確認してみます。

In [34]:
# UnitGroupの一覧を DataFrame で表示する
vcp.df_ugroups()

,vcno,vctype,vcname,state,vcid,cdate
